# 📊 PYUSD Network Dashboard Builder

**Goal**: Provide data pipelines for visualizing:
- Ethereum Network Congestion
- MEV Events involving PYUSD
- PYUSD's Imprint on Ethereum (volume, holders, gas usage)

This notebook prepares data views to be used in **Google Looker Studio**.

In [ ]:
# 🧰 Install libraries
!pip install --quiet google-cloud-bigquery gspread gspread_dataframe google-auth pandas

In [ ]:
# ✅ Imports and auth
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
import gspread
from gspread_dataframe import set_with_dataframe
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)
client = bigquery.Client()
print("🔐 Authenticated and connected to GCP")

In [ ]:
# 📈 Query: Network Congestion Metrics (gas price, tx count, avg block time)
query_congestion = '''
SELECT
  DATE(block_timestamp) AS date,
  COUNT(*) AS tx_count,
  AVG(gas_price) / 1e9 AS avg_gwei,
  AVG(block_number - LAG(block_number) OVER(ORDER BY block_timestamp)) AS avg_block_spacing
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE block_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
GROUP BY date
ORDER BY date DESC
'''
df_congestion = client.query(query_congestion).to_dataframe()

In [ ]:
# 🧠 Query: PYUSD Token Impact
query_pyusd = '''
SELECT
  DATE(block_timestamp) AS date,
  COUNT(*) AS transfers,
  SUM(value / 1e6) AS total_volume_m,
  APPROX_COUNT_DISTINCT(from_address) AS unique_senders,
  SUM(gas_price * gas_used) / 1e18 AS total_gas_eth
FROM `bigquery-public-data.crypto_ethereum.token_transfers`
WHERE token_address = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'
  AND block_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
GROUP BY date
ORDER BY date DESC
'''
df_pyusd = client.query(query_pyusd).to_dataframe()

In [ ]:
# ⚠️ Simulated MEV signal: identify txs with high priority fee vs median
query_mev = '''
SELECT
  DATE(block_timestamp) AS date,
  COUNT(*) AS suspected_mev_events
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE gas_price > 3 * (SELECT AVG(gas_price) FROM `bigquery-public-data.crypto_ethereum.transactions`
                      WHERE block_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY))
AND to_address = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'
GROUP BY date
ORDER BY date DESC
'''
df_mev = client.query(query_mev).to_dataframe()

In [ ]:
# 📤 Export to Google Sheets for Looker Studio
sheet = gc.create("PYUSD_Dashboard_Exports")
ws1 = sheet.get_worksheet(0)
ws1.update_title("Network_Congestion")
set_with_dataframe(ws1, df_congestion)

ws2 = sheet.add_worksheet("PYUSD_Impact", 100, 20)
set_with_dataframe(ws2, df_pyusd)

ws3 = sheet.add_worksheet("MEV_Events", 100, 20)
set_with_dataframe(ws3, df_mev)

print("📊 Data exported to Google Sheets:", sheet.url)